In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [7]:
my $practice_input = qq/
5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
/;


5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0


In [13]:
sub subp { my ($a, $b) = @_; return [ map { $a->[$_] - $b->[$_] } 0 .. $#$a ]; }
sub addp { my ($a, $b) = @_; return [ map { $a->[$_] + $b->[$_] } 0 .. $#$a ]; }
sub mulp { my ($a, $v) = @_; return [ map { $a->[$_] * $v } 0 .. $#$a ]; }
sub stringp { my $p = $_[0] // $_; return join ',', @$p }
sub rotate90p { my $p = $_[0] // $_; return [$p->[1], -$p->[0]] }
sub rotate270p { my $p = $_[0] // $_; return [-$p->[1], $p->[0]] }
sub uniqp { my %h; @h{map { join ',', @$_ } @_} = (); map [ split ',' ], keys %h }
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0 && $p->[1] >= 0 && $p->[0] < $shape[0] && $p->[1] < $shape[1];
}
sub adjacent_points {
    my ($p, @shape) = @_;
    my @adj;
    return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}

In [140]:
sub string_2d_numbers_array {
    my ($arr) = @_;
    return join "\n", map { join '', map { defined ? sprintf "% 6d|", $_ : '      |' } @$_ } @$arr;
}
$graphql::graphql_methods{map_distance} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};
        # say 'wat:', Dumper $self->{arr};
        my @shape = shape($arr);

        state %cache;

        return $cache{$arr}{stringp($self->{coord})} if exists $cache{$arr};

        my @start =
            grep defined,
            map selector('where.coord')->($_),
            flatten_nd
            map_nd_indexed graphql_query(qq/{ where(value:"$params->{from_value}") { coord } }/),
            $arr;
        my %stepped;
        my @next_points = @start;
        my $dist = 0;
        while (@next_points) {
            die if $dist > 2500;
            @stepped{map stringp, @next_points} = ($dist) x @next_points;
            @next_points =
                uniqp
                grep { not exists $stepped{stringp $_} }
                grep $arr->[$_->[0]][$_->[1]] ne '#',
                map adjacent_points($_, @shape),
                @next_points;
            $dist += 1;
        }
        $cache{$arr} = \%stepped;
        return $cache{$arr}{stringp $self->{coord}};
    };
sub process_day18_part1 {
    my ($input, $how_many, @shape) = @_;
    my %points_table = map { $_ => 1 } map { join ',', reverse split ',' } (grep $_ ne '', split "\n", $input)[0 .. $how_many-1];
    my $map = nd_from_shape(@shape);

    my $obstacle_map =
        # string_2d_map_array
        map_nd_indexed sub { my ($arr, $p) = @_; exists $points_table{stringp $p} ? '#' : '.' },
        $map;

    $obstacle_map->[0][0] = '@';
    $obstacle_map->[$shape[0]-1][$shape[1]-1] = 'E';

    say "debug:\n", string_2d_map_array $obstacle_map;

    my $distance_map =
        map_nd selector ('distance'),
        map_nd_indexed graphql_query('{ distance:map_distance(from_value:"@") }'),
        $obstacle_map;

    say string_2d_numbers_array $distance_map;

    return $distance_map->[$shape[0]-1][$shape[1]-1];
}
timer sub {
    # say Dumper process_day18_part1($practice_input, 12, 7, 7);
    confirm process_day18_part1($practice_input, 12, 7, 7), 22;
}

debug:
@..#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#...E
     0|     1|     2|      |     8|     9|    10|
     1|     2|      |     6|     7|      |    11|
     2|     3|     4|     5|      |    13|    12|
     3|     4|     5|      |    15|    14|      |
     4|     5|      |    17|    16|      |    24|
     5|      |    19|    18|      |    22|    23|
      |      |      |    19|    20|    21|    22|
confirmed: 22
[time] 0 seconds


0

Warning: Subroutine string_2d_numbers_array redefined at reply input line 1.

Subroutine process_day18_part1 redefined at reply input line 38.


In [141]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/18/input');
    $res = process_day18_part1($input, 1024, 71, 71);
    say Dumper $res;
    confirm $res, 370;
};

debug:
@....#...#.....#.......#...#.....#.......#.............................
.###.#.#.#.###.#.#.#####.#.#.#.#.#####.#.#..#....#########.#####.......
...#...#...#.#.#.#.......#...#.#.....#.#.#.......#.......#.#...#.......
.#.####.####.#.#.####.###.####.#####.#.#.####....#.#####.#.#.#.#.......
.#.#.......#...#.#...#.....#...#...#.#.#.....#...#.#.......#.#.#.......
.#.#####...#.###.#.#..####.#.##..#.#.#.#####.#####.#.#######...#.......
.#.....#.....#.#...#.#.....#.#...#.....#...#...#...#...#...............
.#####.###.##..#####.#.#.###.#.#########..#.##.#.#######..##.#.#.......
.#...#...#.....#.......#.#...#.#...#.....#.....#.#.....................
.#.#.###.#######.#######.#.#####.#.##.##.#.#####.#.#.#.#.....#.........
.#.#...#.#.......#.#...#.#.......#...#...#.......#.....................
.#.##.##.#.#..#..#.#.#.###.######.##.#..#.######.#.#...................
...#.....#.#.......#.#...#.......#...#.#.........#.....................
##.#.####..#######.#.#.#.#######.#.###.#.########........

0

In [94]:
say post_answer('2024/day/18/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 18 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [144]:
sub is_solvable {
    my ($input, @shape) = @_;
    my %points_table = map { $_ => 1 } map { join ',', reverse split ',' }(grep $_ ne '', split "\n", $input);
    my $map = nd_from_shape(@shape);

    my $obstacle_map =
        # string_2d_map_array
        map_nd_indexed sub { my ($arr, $p) = @_; exists $points_table{stringp $p} ? '#' : '.' },
        $map;

    $obstacle_map->[0][0] = '@';
    $obstacle_map->[$shape[0]-1][$shape[1]-1] = 'E';


    # say "debug:\n", string_2d_map_array $obstacle_map;

    my $distance_map =
        map_nd selector ('distance'),
        map_nd_indexed graphql_query('{ distance:map_distance(from_value:"@") }'),
        $obstacle_map;

    # say string_2d_numbers_array $distance_map;

    return $distance_map->[$shape[0]-1][$shape[1]-1];
}
sub process_day18_part2 {
    my ($input, @shape) = @_;
    my @points = grep $_ ne '', split "\n", $input;
    my $bottom = 0;
    my $top = $#points;

    my $steps = 0;
    while ($top - $bottom > 1) {
        last if $steps++ > 100;
        my $mid = int(($top + $bottom) / 2);
        say "trying: $bottom < $mid < $top";
        if (is_solvable(join("\n", @points[0 .. $mid]), @shape)) {
            $bottom = $mid;
        } else {
            $top = $mid;
        }
        # warn "huh: $top $bottom";
    }
    # die 'wat';
    say "debug: bottom: $bottom is_solvable? ", is_solvable(join("\n", @points[0 .. $bottom]), @shape);
    say "point ($points[$bottom])";
    say "debug: top: $top is_solvable? ", is_solvable(join("\n", @points[0 .. $top]), @shape) // '';
    say "point ($points[$top])";

    # die unless defined is_solvable(join("\n", @points[0 .. $bottom]), @shape);
    # die if defined is_solvable(join("\n", @points[0 .. $top]), @shape);

    return $points[$top];
}
timer sub {
    say Dumper process_day18_part2($practice_input, 7, 7);
    confirm process_day18_part2($practice_input, 7, 7), '6,1';
    
    # my $input = get_challenge('2024/day/18/input');
    # $res = process_day18_part2($input, 71, 71);
}

trying: 0 < 12 < 24
trying: 12 < 18 < 24
trying: 18 < 21 < 24
trying: 18 < 19 < 21
trying: 19 < 20 < 21
debug: bottom: 19 is_solvable? 24
point (1,1)
debug: top: 20 is_solvable? 
point (6,1)
$VAR1 = '6,1';

trying: 0 < 12 < 24
trying: 12 < 18 < 24
trying: 18 < 21 < 24
trying: 18 < 19 < 21
trying: 19 < 20 < 21
debug: bottom: 19 is_solvable? 24
point (1,1)
debug: top: 20 is_solvable? 
point (6,1)
confirmed: 6,1
[time] 0 seconds


0

Warning: Subroutine is_solvable redefined at reply input line 1.

Subroutine process_day18_part2 redefined at reply input line 26.


In [145]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/18/input');
    $res = process_day18_part2($input, 71, 71);
    say Dumper $res;
    # confirm $res, '1,0,2,0,5,7,2,1,3';
};

trying: 0 < 1724 < 3449
trying: 1724 < 2586 < 3449
trying: 2586 < 3017 < 3449
trying: 3017 < 3233 < 3449
trying: 3017 < 3125 < 3233
trying: 3017 < 3071 < 3125
trying: 3017 < 3044 < 3071
trying: 3017 < 3030 < 3044
trying: 3030 < 3037 < 3044
trying: 3030 < 3033 < 3037
trying: 3030 < 3031 < 3033
trying: 3031 < 3032 < 3033
debug: bottom: 3031 is_solvable? 668
point (23,58)
debug: top: 3032 is_solvable? 
point (65,6)
$VAR1 = '65,6';

[time] 2 seconds


2

In [147]:
say post_answer('2024/day/18/answer', 2, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 18 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1